In [ ]:
%matplotlib inline
import flopy
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
import pandas as pd
import pathlib as pl
import pickle
import xarray as xa

In [ ]:
sample_frequency = "monthly"  # monthly or annual
name = "sv"

In [ ]:
# load the simple model setup
base_ws = pl.Path(f"../models/synthetic-valley-base-{sample_frequency}")
advanced_ws = base_ws.parent / f"synthetic-valley-opt-{sample_frequency}"

In [ ]:
# unit conversion factor
in2ft = 1.0 / 12.0

## Load the appropriate temporal data

In [ ]:
idx_end_calibration = 0
if sample_frequency == "monthly":
    idx_end_period2 = 120
    idx_end_period3 = 240
elif sample_frequency == "annual":
    idx_end_period2 = 10
    idx_end_period3 = 20
else:
    raise ValueError(f"invalid sample_frequency: '{sample_frequency}'")

In [ ]:
path = pl.Path(f"../synthetic-valley/data/temporal_data_{sample_frequency}.parquet")
temporal_df = pd.read_parquet(path)

In [ ]:
temporal_df

## Define the stress period data

In [ ]:
start_date = pd.to_datetime("1962-01-01 00:00:00")
start_date_time = str(start_date).replace(" ", "T")

end_calibration = temporal_df.index[idx_end_calibration]
end_period_two = temporal_df.index[idx_end_period2]
end_period_three = temporal_df.index[idx_end_period2]

end_periods = [end_calibration, end_period_two, end_period_three]
end_periods

In [ ]:
totim_end = [float((end_calibration - start_date).days)]
totim_end += [float((end_period_two - start_date).days)]
totim_end += [float((end_period_three - start_date).days)]
totim_end

## Spatial data for the model

In [ ]:
nc_path = pl.Path("../synthetic-valley/data/synthetic_valley_truth.nc")
nc_ds = xa.open_dataset(nc_path)
lake_location = nc_ds["lake_location"].to_numpy()
lake_area = float(lake_location.sum()) * 500.0 * 500.0

## Load the existing base model

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=base_ws, write_headers=False)

In [ ]:
nper = sim.tdis.nper.array
gwf = sim.get_model(name)
nlay, nrow, ncol = gwf.dis.nlay.array, gwf.dis.nrow.array, gwf.dis.ncol.array
shape2d = (nrow, ncol)
shape3d = (nlay, nrow, ncol)

In [ ]:
izone = np.zeros(shape3d, dtype=int)
idx = lake_location == 1
izone[0][idx] = 1

In [ ]:
botm = gwf.dis.botm.array

In [ ]:
# list the packages in the gwf model
gwf.get_package_list()

### Change the simulation workspace

In [ ]:
sim.set_sim_path(advanced_ws)

## Add UZF

In [ ]:
# set isUZF to True if UZF package has been added

isUZF = True

In [ ]:
# use the vertical hydraulic conductivity to define vks for the uzf package

k33 = gwf.npf.k33.array

In [ ]:
# create uzf packagedata
# <ifno> <cellid> <landflag> <ivertcon> <surfdep> <vks> <thtr> <thts> <thti> <eps> [<boundname>]

packagedata = []
ifno = 0
for i in range(nrow):
    for j in range(ncol):
        if lake_location[i, j] == 1:
            continue
        cellid = (0, i, j)
        packagedata.append(
            (
                ifno,
                cellid,
                1,
                0,
                1.0,
                k33[cellid],
                0.05,
                0.25,
                0.1,
                3.5,
            )
        )

        ifno += 1

In [ ]:
# uzf stress period data with rainfall and land evapotranspiration
# <ifno> <finf> <pet> <extdp> <extwc> <ha> <hroot> <rootact> [<aux(naux)>]

uzf_spd = {}
rain_tag = "PRCP (Inches)"
pet_tag = "land et (inches)"
for n in range(nper):
    row = temporal_df.iloc[n]
    rain = float(row[rain_tag]) * in2ft
    pet = float(row[pet_tag]) * in2ft
    spd = []
    for values in packagedata:
        ifno, cellid = values[:2]
        spd.append(
            (
                ifno,
                rain,
                pet,
                10.0,
                -999.0,
                -999.0,
                -999.0,
                -999.0,
            )
        )
    uzf_spd[n] = spd

In [ ]:
# add uzf package to the gwf simulation
# set mover to True, simulated_et to True, and name the package "UZF-1"

uzf = flopy.mf6.ModflowGwfuzf(
    gwf,
    pname="UZF-1",
    mover=True,
    simulate_et=True,
    nuzfcells=len(packagedata),
    packagedata=packagedata,
    perioddata=uzf_spd,
)

In [ ]:
# remove the existing recharge package

gwf.remove_package("RCH_0")

# Convert the spring drain to a MAW well

In [ ]:
pak = gwf.get_package("SPRING")
spd = pak.stress_period_data.get_data(0)
spd

In [ ]:
# create maw packagedata
# <ifno> <radius> <bottom> <strt> <condeqn> <ngwfnodes> [<aux(naux)>] [<boundname>]
# set the radius to 0.5 and use the "thiem" conductance equation
radius = 25.0
cellids = spd["cellid"]
k0, i, j = cellids[0]
k1, i, j = cellids[1]
zelevs = [
    float(botm[k0 - 1, i, j]),
    float(botm[k0, i, j]),
    float(botm[k1, i, j]),
]
package_data = [(0, radius, zelevs[-1] - 0.5, 0.0, "thiem", 2, "leake_spring")]
package_data

In [ ]:
# create maw connectiondata
# <ifno> <icon> <cellid> <scrn_top> <scrn_bot> <hk_skin> <radius_skin>

connection_data = [
    (0, 0, (k0, i, j), zelevs[0], zelevs[1], -999.0, -999.0),
    (0, 1, (k1, i, j), zelevs[1], zelevs[2], -999.0, -999.0),
]
connection_data

In [ ]:
gwf.remove_package("SPRING")

In [ ]:
fwelev = float(spd["elev"][0])
fwcond = float(spd["cond"].sum())  # * 1000.0
fwrlen = 1.0
spring_spd = [(0, "flowing_well", fwelev, fwcond, fwrlen)]
spring_spd

In [ ]:
maw_spring = flopy.mf6.ModflowGwfmaw(
    gwf,
    flowing_wells=True,
    mover=True,
    boundnames=True,
    nmawwells=1,
    packagedata=package_data,
    connectiondata=connection_data,
    perioddata=spring_spd,
    pname="spring",
    filename=f"{gwf.name}.spring.maw",
)

In [ ]:
# create spring observations

spring_obs = [
    ("SPRING-FLOW", "FW-TO-MVR", "LEAKE_SPRING"),
]
spring_obs = {
    f"{gwf.name}.spring.obs.csv": spring_obs,
}

In [ ]:
# initialize the spring observations

maw_spring.obs.initialize(
    filename=f"{gwf.name}.spring.obs",
    print_input=True,
    continuous=spring_obs,
)

## Convert RIV package to SFR package

In [ ]:
# set isSFR to True if SFR package has been added

isSFR = True

In [ ]:
# get the existing river package

pak = gwf.get_package("RIV-1")

In [ ]:
# get the river stress period data for stress period 1
# this will be used to define some sfr package data
# set nreaches to the length of the stress period data array

spd = pak.stress_period_data.get_data(0)
nreaches = len(spd)

In [ ]:
for cellid, stage, cond, rbot, boundname in spd:
    izone[cellid] = 2

In [ ]:
# calculate the bed gradient for each reach

rgrad = np.diff(spd["rbot"]) * (-1.0)
rgrad = np.array([float(rgrad[0])] + rgrad.tolist())

In [ ]:
# create sfr packagedata and connection data
# <ifno> <cellid> <rlen> <rwid> <rgrd> <rtp> <rbth> <rhk> <man> <ncon> <ustrf> <ndv> [<aux(naux)>] [<boundname>]

delc = gwf.dis.delc.array
rwid = 30.0
rbth = 1.0
man = 0.03
ustrf = 1.0
ndv = 0
packagedata = []
connectiondata = []

for ifno, (cellid, stage, cond, rbot, boundname) in enumerate(spd):
    rconn = [ifno]
    ncon = 1
    if ifno > 0 and ifno < nreaches - 1:
        ncon += 1
    if ifno > 0:
        rconn.append(ifno - 1)
    if ifno < nreaches - 1:
        rconn.append(-(ifno + 1))
    connectiondata.append(rconn)
    rlen = float(delc[cellid[1]])
    rhk = float(cond) / (rwid * rlen)
    packagedata.append(
        (
            ifno,
            cellid,
            rlen,
            rwid,
            float(rgrad[ifno]),
            float(rbot),
            rbth,
            rhk,
            man,
            ncon,
            ustrf,
            ndv,
            boundname,
        )
    )

In [ ]:
# create sfr observations

sfr_obs = [
    ("RIV-SWGW", "SFR", "RIV"),
    ("RIV-FLOW", "downstream-flow", nreaches - 1),
]
sfr_obs = {
    f"{name}.sfr.obs.csv": sfr_obs,
}

In [ ]:
# add sfr package to the gwf simulation
# set time_conversion to 86400.0, mover to True, boundname to True, and name the package "SFR-1"

sfr = flopy.mf6.ModflowGwfsfr(
    gwf,
    time_conversion=86400.0,
    save_flows=True,
    mover=True,
    boundnames=True,
    nreaches=nreaches,
    packagedata=packagedata,
    connectiondata=connectiondata,
    pname="SFR-1",
)

In [ ]:
# initialize the sfr observations

sfr.obs.initialize(
    filename=f"{name}.sfr.obs",
    print_input=True,
    continuous=sfr_obs,
)

In [ ]:
# remove the existing riv and river observation packages

if isSFR:
    for pak_name in ("RIV-1", "RIV_OBS"):
        gwf.remove_package(pak_name)

## Convert high-K lake to LAK package

In [ ]:
# set islLake to True if LAK package has been added

isLake = True

In [ ]:
# # reset k and k33 in lake to k and k33 in layer 2

# k11 = gwf.npf.k.array
# k33 = gwf.npf.k33.array
# k11[0] = k11[1]
# k33[0] = k33[1]
# gwf.npf.k = k11
# gwf.npf.k33 = k33

In [ ]:
# create 3d lake_array array and set to 0 in lake location and -1 elsewhere

lake_array = np.full(shape3d, -1, dtype=int)
idx = lake_location == 1
lake_array[0, :, :][idx] = 0

In [ ]:
# use get_lak_connections to create lake data and set bedleak to 1e-2

idomain, lake_connection_dict, lake_connection_data = (
    flopy.mf6.utils.get_lak_connections(
        gwf.modelgrid,
        lake_array,
        bedleak=1e-2,
    )
)

In [ ]:
# set model idomain to new idomain

gwf.dis.idomain = idomain

In [ ]:
# create lake package data using value to define nlakeconn and set boundname to "lake_harbaugh"
# <ifno> <strt> <nlakeconn> [<aux(naux)>] [<boundname>]

lake_pakagedata = []
for key, value in lake_connection_dict.items():
    lake_pakagedata.append((int(key), 13.0, value, "lake_harbaugh"))
nlakes = len(lake_pakagedata)
lake_pakagedata

In [ ]:
# create lake observations

lake_obs = {
    f"{name}.lake.obs.csv": [
        ("LAKE-STAGE", "STAGE", "LAKE_HARBAUGH"),
        ("LAKE-SWGW", "LAK", "LAKE_HARBAUGH"),
    ]
}

In [ ]:
# create lake stress period data with rainfall and evaporation

lake_spd = {}
rain_tag = "PRCP (Inches)"
pet_tag = "lake et (inches)"
for n in range(nper):
    row = temporal_df.iloc[n]
    rain = float(row[rain_tag]) * in2ft
    pet = float(row[pet_tag]) * in2ft
    lake_spd[n] = [(0, "rainfall", rain), (0, "evaporation", pet)]

In [ ]:
# add lak package to the gwf simulation
# set mover to True, boundnames to True, and name the package "LAK-1"

lak = flopy.mf6.ModflowGwflak(
    gwf,
    boundnames=True,
    print_input=True,
    print_stage=True,
    mover=True,
    pname="LAK-1",
    stage_filerecord=f"{name}.lak.hds",
    nlakes=nlakes,
    packagedata=lake_pakagedata,
    connectiondata=lake_connection_data,
    perioddata=lake_spd,
)

In [ ]:
# initialize the lake observations

lak.obs.initialize(
    filename=f"{name}.lak.obs",
    print_input=True,
    continuous=lake_obs,
)

In [ ]:
# redefine existing groundwater flow observation file to remove the existing "lake" observations

if isLake:
    pak = gwf.get_package("GWF-OBS")

    # create a new groundwater observation dictionary
    gwf_obs = {}
    for key, value in pak.continuous.get_active_key_list():
        if key != "sv.lake.obs.csv":
            gwf_obs[key] = value.get_data().tolist()

    # remove the existing groundwater observation package
    gwf.remove_package("GWF-OBS")

    # create a new
    gwf_obs = flopy.mf6.ModflowUtlobs(gwf, print_input=True, continuous=gwf_obs)

In [ ]:
# remove the existing EVT package

if isLake:
    gwf.remove_package("EVT_0")

## Add Mover

In [ ]:
perioddata = []
packages = []
if isUZF:
    packages = [["uzf-1"], ["LAK-1"], ["SFR-1"]]
    for values in uzf.packagedata.array:
        ifno, cellid = values["ifno"], values["cellid"]
        i = cellid[1]
        if i < 20:
            dst = "LAK-1"
        else:
            dst = "SFR-1"
        perioddata.append(("uzf-1", int(ifno), dst, 0, "factor", 0.10))
if len(packages) < 1:
    packages.append(["SFR-1"])
packages.append(["SPRING"])
perioddata.append(("SPRING", 0, "SFR-1", 0, "factor", 1.0))

maxmvr = len(perioddata)

perioddata_dict = {
    0: perioddata,
}

perioddata_dict[0]

In [ ]:
mvr = flopy.mf6.ModflowGwfmvr(
    gwf,
    maxmvr=maxmvr,
    maxpackages=len(packages),
    packages=packages,
    perioddata=perioddata_dict,
)

## Write and run simulation

In [ ]:
# gwf.set_all_data_external(external_data_folder="external")

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

## Plot the results

### Model Properties

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Hydraulic conductivity")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(gwf.npf.k.array[idx], masked_values=[2000000.0])
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Bottom Elevation")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(gwf.dis.botm.array[idx])
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Cell thickness")
    z = gwf.modelgrid.cell_thickness

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(z[idx])
        ax.set_title(f"Layer {idx + 1}")

### Simulated Heads and Drawdown

In [ ]:
def get_heads(totim):
    hds = gwf.output.head().get_data(totim=totim)
    if isLake:
        stage = gwf.lak.output.stage().get_data(totim=totim)
        h1 = hds[0]
        stage = np.full(shape2d, stage[0], dtype=float)
        idx = lake_location == 1
        h1[idx] = stage[idx]
        hds[0, :, :] = h1
    return hds

In [ ]:
levels = np.arange(2, 20.0, 2)

#### Calibration

In [ ]:
totim = totim_end[0]
hds = get_heads(totim)

In [ ]:
v = nc_ds["head_layer2"].values
v.min(), v.max(), v.mean()

In [ ]:
for k in range(nlay):
    diff = hds[k] - nc_ds[f"head_layer{k + 1}"].values
    print(f"{k + 1}: {diff.min()} {diff.max()} {diff.mean()}")

In [ ]:
if sample_frequency == "monthly":
    for k in range(nlay):
        nc_ds[f"head_layer{k + 1}"].values = hds[k]
        nc_ds[f"izone{k + 1}"] = (("y", "x"), izone[k])
    temp_path = nc_path.parent / "temp.nc"
    nc_ds.to_netcdf(temp_path)

    temp_path.rename(nc_path)

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Calibration")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
v = gwf.output.budget()
v.get_unique_record_names()

In [ ]:
v = gwf.output.budget()
bytearray("SFR", "ascii") in v.get_unique_record_names()

In [ ]:
v.get_unique_record_names()

In [ ]:
v = np.array(gwf.output.budget().get_unique_record_names()).astype(str)
v, "             SFR" in v

In [ ]:
if isSFR:
    riv_text = "sfr"
else:
    riv_text = "riv"
v = gwf.output.budget().get_data(text=riv_text, totim=totim)[0]["q"]
print(f"River infiltration: {np.all(v > 0)}\n{v}")

##### Calculate the residuals

In [ ]:
obs_path = pl.Path("../synthetic-valley/data")
with open(obs_path / "obs_data.pkl", "rb") as f:
    obs_rc_locs, well_depth, aq_layer = pickle.load(f)

cal_loc_wt = [(0, i, j) for i, j in obs_rc_locs]
cal_loc_aq = [(aq_layer[idx], i, j) for idx, (i, j) in enumerate(obs_rc_locs)]

In [ ]:
wt_obs = []
aq_layer = []
aq_obs = []
for idx, (i, j) in enumerate(obs_rc_locs):
    iloc = (i, j)
    tag = "head_layer1"
    wt_obs.append(float(nc_ds[tag].values[iloc]))
    wz = well_depth[idx]
    zcell = np.array(botm)[:, i, j]
    klay = 0
    for kk in range(1, nlay):
        z0 = zcell[kk - 1]
        z1 = zcell[kk]
        if wz < z0 and wz >= z1:
            klay = kk
            break
    tag = f"head_layer{klay + 1}"
    aq_layer.append(klay)
    aq_obs.append(float(nc_ds[tag].values[iloc]))

In [ ]:
sim_wt = np.array([hds[idx] for idx in cal_loc_wt])

In [ ]:
resid_wt = sim_wt - np.array(wt_obs)
resid_wt

In [ ]:
sim_aq = np.array([hds[idx] for idx in cal_loc_aq])

In [ ]:
resid_aq = sim_aq - np.array(aq_obs)
resid_aq

In [ ]:
resid_gb = np.concatenate((resid_wt, resid_aq))

In [ ]:
print(
    f"Water Table Statistics\nMean Error: {resid_wt.mean()} ft.\nRMSE:       {np.sqrt((resid_wt**2).sum()) / resid_wt.shape[0]} ft."
)

In [ ]:
print(
    f"Lower Aquifer Statistics\nMean Error: {resid_aq.mean()} ft.\nRMSE:       {np.sqrt((resid_aq**2).sum()) / resid_aq.shape[0]} ft."
)

In [ ]:
print(
    f"Global Statistics\nMean Error: {resid_gb.mean()} ft.\nRMSE:       {np.sqrt((resid_gb**2).sum()) / resid_gb.shape[0]} ft."
)

##### Plot the residuals

In [ ]:
xy = [
    (float(gwf.modelgrid.xcellcenters[i, j]), float(gwf.modelgrid.ycellcenters[i, j]))
    for i, j in obs_rc_locs
]

In [ ]:
x, y = np.array(xy)[:, 0], np.array(xy)[:, 1]

In [ ]:
grid_x, grid_y = np.meshgrid(gwf.modelgrid.xycenters[0], gwf.modelgrid.xycenters[1])

In [ ]:
# Linearly interpolate the data (x, y) on a grid defined by (xi, yi).
triang = tri.Triangulation(x, y)

In [ ]:
interpolator = tri.LinearTriInterpolator(triang, resid_wt)
grid_resid_wt = interpolator(grid_x, grid_y)

In [ ]:
interpolator = tri.LinearTriInterpolator(triang, resid_aq)
grid_resid_aq = interpolator(grid_x, grid_y)

In [ ]:
resid_levels = np.arange(-2, 2.25, 0.25)

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 2, figsize=(8, 5), sharey=True)
    fig.suptitle("Residuals")

    ax = axs[0]
    ax.set_xlim(0, 12500)
    ax.set_ylim(0, 20000)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    mm.plot_array(lake_location, cmap="Blues_r", masked_values=[0])
    mm.plot_grid(lw=0.5, color="0.5")
    ax.scatter(x, y, s=3, c="black")
    for i, txt in enumerate(resid_wt):
        ax.annotate(f"{txt:.2f}", (x[i], y[i]))
    cs = ax.contour(
        grid_x,
        grid_y,
        grid_resid_wt,
        levels=resid_levels,
        linewidths=0.75,
        colors="red",
    )
    plt.clabel(cs, inline=True, fontsize=8)
    ax.set_title("Water Table")

    ax = axs[1]
    ax.set_xlim(0, 12500)
    ax.set_ylim(0, 20000)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    mm.plot_grid(lw=0.5, color="0.5")
    ax.scatter(x, y, s=3, c="black")
    for i, txt in enumerate(resid_aq):
        ax.annotate(f"{txt:.2f}", (x[i], y[i]), clip_on=False)
    cs = ax.contour(
        grid_x,
        grid_y,
        grid_resid_aq,
        levels=resid_levels,
        linewidths=0.75,
        colors="red",
    )
    plt.clabel(cs, inline=True, fontsize=8)
    ax.set_title("Lower Aquifer")

#### First 10 year transient period

In [ ]:
totim = totim_end[0]
totim1 = totim_end[1]

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Transient Period 1")
    hds = get_heads(totim1)

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Drawdown - Transient Period 1")
    ddn = get_heads(totim) - get_heads(totim1)

    ddn_max = ddn[:, 16, :].max()

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(ddn)
        cs = mm.contour_array(ddn, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
print(f"Maximum Drawdown: {ddn_max}")

In [ ]:
v = gwf.output.budget().get_data(text=riv_text, totim=totim1)[0]["q"]
print(f"Induced river infiltration: {np.all(v > 0)}\n{v}")

#### Second 10 year transient period

In [ ]:
totim2 = totim_end[2]

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Transient Period 2 - Prediction")
    hds = get_heads(totim2)

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Drawdown - Transient Period 2 - Prediction")
    ddn = get_heads(totim) - get_heads(totim2)

    ddn_max = ddn[:, 16, :].max()

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(ddn)
        cs = mm.contour_array(ddn, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
print(f"Maximum Drawdown: {ddn_max}")

In [ ]:
v = gwf.output.budget().get_data(text=riv_text, totim=totim2)[0]["q"]
print(f"Induced river infiltration: {np.all(v > 0)}\n{v}")

### Spring flow

In [ ]:
df = gwf.spring.output.obs().get_dataframe(start_datetime=start_date)
df["SPRING-FLOW"] /= -86400

In [ ]:
Q0 = df["SPRING-FLOW"].iloc[0]
df["PCT_DIFF"] = -100.0 * (df["SPRING-FLOW"] - Q0) / Q0
df

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(2, 1, figsize=(9, 3), sharex=True)

    fig.suptitle("Leake Spring")

    ax = axs[0]
    # ax.set_ylim(-5, 25)
    df["SPRING-FLOW"].plot(ax=ax, ls="-", marker="o", clip_on=False)
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("Spring\nDischarge, cfs")

    ax = axs[1]
    ax.set_ylim(-100, 100)
    df["PCT_DIFF"].plot(ax=ax, ls="-", marker="o", clip_on=False)
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("Reduction\n in Spring\nDischarge, %")
    ax.set_xlabel("Stress Period")

    fig.align_labels()

### Streamflow results

In [ ]:
if isSFR:
    df = gwf.sfr.output.obs().get_dataframe(start_datetime=start_date)
    df["RIV-FLOW"] /= -86400
else:
    df = gwf.riv.output.obs().get_dataframe()

df["RIV-SWGW"] /= -86400
df["TOTAL"] = df["RIV-SWGW"]

Q0 = df["TOTAL"].iloc[0]
df["PCT_DIFF"] = -100.0 * (df["TOTAL"] - Q0) / Q0
df

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(2, 1, figsize=(9, 3), sharex=True)

    fig.suptitle("Southern Boundary - Gage 1")

    ax = axs[0]
    ax.set_ylim(-5, 25)
    df["RIV-FLOW"].plot(ax=ax, ls="-", marker="o", clip_on=False)
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("River\nDischarge, cfs")

    ax = axs[1]
    ax.set_ylim(-100, 100)
    df["PCT_DIFF"].plot(ax=ax, ls="-", marker="o", clip_on=False)
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("Reduction\n in River\nDischarge, %")
    ax.set_xlabel("Stress Period")

    fig.align_labels()

### Lake stage

In [ ]:
if isLake:
    fpth = advanced_ws / f"{name}.lake.obs.csv"
else:
    fpth = advanced_ws / f"{name}.gwf.scenario.csv"

In [ ]:
lake_df = flopy.utils.Mf6Obs(fpth).get_dataframe(start_datetime=start_date)

In [ ]:
if isLake:
    lake_df["LAKE-SWGW"] *= 12.0 / lake_area

In [ ]:
lake_df

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(1, 1, figsize=(9, 1.5))

    lake_df["LAKE-STAGE"].plot(
        ax=ax,
        ls="-",
        marker="o",
        clip_on=False,
    )
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("Lake\nStage, ft")
    ax.set_xlabel("Stress Period")
    ax.set_ylim(10, 15)